In [15]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
from torchvision.utils import save_image
import os
import time

# create directory for save result_img

In [16]:
if not os.path.exists('./dc_img'):
    os.mkdir('./dc_img')

# define function to create thumbnail

In [17]:
def to_img(x):
    out = 0.5 * (x + 1)
    out = out.clamp(0, 1)
    out = out.view(-1, 1, 28, 28)
    return out

# initialization

In [18]:
batch_size = 128
num_epoch = 100
z_dimension = 100  # noise dimension

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [20]:
# mnist = datasets.MNIST('./mnist_data', transform=img_transform)
dataloader = DataLoader( datasets.MNIST("./mnist_data", train=True, download=True,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
#                               transforms.Normalize((0.1307,), (0.3081,))
                          ])),
    batch_size=batch_size, shuffle=True,
    num_workers=4, pin_memory=True)

# define models

In [21]:
class discriminator(nn.Module):
    def __init__(self):
        super(discriminator, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, 5, padding=2),  # batch, 32, 28, 28
            nn.LeakyReLU(0.2, True),
            nn.AvgPool2d(2, stride=2),  # batch, 32, 14, 14
            )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, 5, padding=2),  # batch, 64, 14, 14
            nn.LeakyReLU(0.2, True),
            nn.AvgPool2d(2, stride=2)  # batch, 64, 7, 7
        )
        self.fc = nn.Sequential(
            nn.Linear(64*7*7, 1024),
            nn.LeakyReLU(0.2, True),
            nn.Linear(1024, 1),
            nn.Sigmoid()
        )
 
    def forward(self, x):
        '''
        x: batch, width, height, channel=1
        '''
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [22]:
class generator(nn.Module):
    def __init__(self, input_size, num_feature):
        super(generator, self).__init__()
        self.fc = nn.Linear(input_size, num_feature)  # batch, 3136=1x56x56
        self.br = nn.Sequential(
            nn.BatchNorm2d(1),
            nn.ReLU(True)
        )
        self.downsample1 = nn.Sequential(
            nn.Conv2d(1, 50, 3, stride=1, padding=1),  # batch, 50, 56, 56
            nn.BatchNorm2d(50),
            nn.ReLU(True)
        )
        self.downsample2 = nn.Sequential(
            nn.Conv2d(50, 25, 3, stride=1, padding=1),  # batch, 25, 56, 56
            nn.BatchNorm2d(25),
            nn.ReLU(True)
        )
        self.downsample3 = nn.Sequential(
            nn.Conv2d(25, 1, 2, stride=2),  # batch, 1, 28, 28
            nn.Tanh()
        )
 
    def forward(self, x):
        x = self.fc(x)
        x = x.view(x.size(0), 1, 56, 56)
        x = self.br(x)
        x = self.downsample1(x)
        x = self.downsample2(x)
        x = self.downsample3(x)
        return x

In [23]:
D = discriminator().to(device)  # discriminator model
G = generator(z_dimension, 3136).to(device)  # generator model

# define optimizer

In [24]:
criterion = nn.BCELoss()  # binary cross entropy
 
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0003)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0003)

# start training

In [ ]:
for epoch in range(num_epoch):
    since = time.time()
    for i, (data, target) in enumerate(dataloader):
        num_img = data.size(0)
        data, target = data.to(device), target.to(device)
        
        real_label = torch.ones(num_img, 1).to(device)
        fake_label = torch.zeros(num_img, 1).to(device)

        pred_real = D(data)
        d_loss_real = criterion(pred_real, real_label)
        real_scores = pred_real  # closer to 1 means better
 
        # compute loss of fake_img
        z = torch.randn(num_img, z_dimension).to(device)
        fake_data = G(z)
        pred_fake = D(fake_data)
        d_loss_fake = criterion(pred_fake, fake_label)
        fake_scores = pred_fake  # closer to 0 means better
 
        # bp and optimize
        d_loss = d_loss_real + d_loss_fake
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
 
        # ===============train generator
        # compute loss of fake_img
        z = torch.randn(num_img, z_dimension).to(device)
        fake_img = G(z)
        output = D(fake_img)
        g_loss = criterion(output, real_label)
 
        # bp and optimize
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
 
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], d_loss: {:.6f}, g_loss: {:.6f} '
                  'D real: {:.6f}, D fake: {:.6f}'
                  .format(epoch, num_epoch, d_loss.item(), g_loss.item(),
                          real_scores.data.mean(), fake_scores.data.mean()))
    print('Epoch[{}/100], time_consumed:{:.6f}'.format(epoch,time.time()-since))
    if epoch == 0:
        real_images = to_img(data.cpu().data)
        save_image(real_images, './dc_img/real_images.png')
    
    fake_images = to_img(fake_data.cpu().data)
    save_image(fake_images, './dc_img/fake_images-{}.png'.format(epoch+1))

Epoch [0/100], d_loss: 0.104139, g_loss: 3.805350 D real: 0.934160, D fake: 0.029623
Epoch [0/100], d_loss: 0.100255, g_loss: 5.486824 D real: 0.960277, D fake: 0.041238
Epoch [0/100], d_loss: 0.296275, g_loss: 2.428895 D real: 0.885859, D fake: 0.107381
Epoch [0/100], d_loss: 0.356639, g_loss: 3.623286 D real: 0.831749, D fake: 0.073557
Epoch[0/100], time_consumed:42.219444
Epoch [1/100], d_loss: 0.294025, g_loss: 2.993270 D real: 0.970351, D fake: 0.186456
Epoch [1/100], d_loss: 0.203311, g_loss: 5.246939 D real: 0.949593, D fake: 0.100236
Epoch [1/100], d_loss: 0.204210, g_loss: 4.913994 D real: 0.922016, D fake: 0.065466
Epoch [1/100], d_loss: 0.215171, g_loss: 4.303040 D real: 0.898790, D fake: 0.050127
Epoch[1/100], time_consumed:40.402043
Epoch [2/100], d_loss: 0.151447, g_loss: 3.929520 D real: 0.963311, D fake: 0.086112
Epoch [2/100], d_loss: 0.201356, g_loss: 3.913198 D real: 0.966426, D fake: 0.102792
Epoch [2/100], d_loss: 0.161651, g_loss: 4.443675 D real: 0.925859, D fake